In [18]:
import re
import pandas as pd
import numpy as np
import math
import nltk
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import make_scorer, accuracy_score, classification_report
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [19]:
file_path = 'C:/Users/Admin/Desktop/'

In [20]:
train = pd.read_csv(file_path+"train.csv")
val = pd.read_csv(file_path+"val.csv")
test = pd.read_csv(file_path+"test.csv")
train

,Unnamed: 0,ID,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,0,2017-En-21441,“Worry is a down payment on a problem you may ...,False,True,False,False,False,False,True,False,False,False,True
1,1,2017-En-31535,Whatever you decide to do make sure it makes y...,False,False,False,False,True,True,True,False,False,False,False
2,2,2017-En-21068,@Max_Kellerman it also helps that the majorit...,True,False,True,False,True,False,True,False,False,False,False
3,3,2017-En-31436,Accept the challenges so that you can literall...,False,False,False,False,True,False,True,False,False,False,False
4,4,2017-En-22195,My roommate: it's okay that we can't spell bec...,True,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6833,833,2017-En-21383,@nicky57672 Hi! We are working towards your hi...,False,False,False,False,False,False,False,False,False,False,False
6834,834,2017-En-41441,@andreamitchell said @berniesanders not only d...,False,True,False,False,False,False,False,False,False,True,False
6835,835,2017-En-10886,@isthataspider @dhodgs i will fight this guy! ...,True,False,True,False,False,False,False,True,False,False,False
6836,836,2017-En-40662,i wonder how a guy can broke his penis while h...,False,False,False,False,False,False,False,False,False,True,False


**Data Preprocessing**

In [21]:
import nltk
nltk.download('stopwords')
import re
from nltk.corpus import stopwords

def remove_tags_and_clean(string):
    removelist = ""
    
    # Remove HTML tags
    result = re.sub(r'<.*?>', '', string)
    
    # Remove URLs
    result = re.sub(r'https://.*', '', result)
    
    # Remove non-alphanumeric characters and replace with spaces
    result = re.sub(r'[^a-zA-Z0-9]+', ' ', result)
    
    # Remove leading and trailing spaces
    result = result.strip()
    
    # Convert to lowercase
    result = result.lower()
    
    return result

# Example usage
train['Tweet'] = train['Tweet'].apply(lambda cw: remove_tags_and_clean(cw))
val['Tweet'] = val['Tweet'].apply(lambda cw: remove_tags_and_clean(cw))
test['Tweet'] = test['Tweet'].apply(lambda cw: remove_tags_and_clean(cw))

# Remove stopwords
stop_words = set(stopwords.words('english'))
train['Tweet'] = train['Tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
val['Tweet'] = val['Tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
test['Tweet'] = test['Tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
train

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Unnamed: 0,ID,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,0,2017-En-21441,worry payment problem may never joyce meyer mo...,False,True,False,False,False,False,True,False,False,False,True
1,1,2017-En-31535,whatever decide make sure makes happy,False,False,False,False,True,True,True,False,False,False,False
2,2,2017-En-21068,max kellerman also helps majority nfl coaching...,True,False,True,False,True,False,True,False,False,False,False
3,3,2017-En-31436,accept challenges literally even feel exhilara...,False,False,False,False,True,False,True,False,False,False,False
4,4,2017-En-22195,roommate okay spell autocorrect terrible first...,True,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6833,833,2017-En-21383,nicky57672 hi working towards highlighted conc...,False,False,False,False,False,False,False,False,False,False,False
6834,834,2017-En-41441,andreamitchell said berniesanders play hrc cam...,False,True,False,False,False,False,False,False,False,True,False
6835,835,2017-En-10886,isthataspider dhodgs fight guy insult lions li...,True,False,True,False,False,False,False,True,False,False,False
6836,836,2017-En-40662,wonder guy broke penis sex serious,False,False,False,False,False,False,False,False,False,True,False


In [22]:
nltk.download('wordnet')
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    st = ""
    for w in w_tokenizer.tokenize(text):
        st = st + lemmatizer.lemmatize(w) + " "
    return st
train['Tweet'] = train['Tweet'].apply(lemmatize_text)
val['Tweet'] = val['Tweet'].apply(lemmatize_text)
test['Tweet'] = test['Tweet'].apply(lemmatize_text)
train

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Unnamed: 0,ID,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,0,2017-En-21441,worry payment problem may never joyce meyer mo...,False,True,False,False,False,False,True,False,False,False,True
1,1,2017-En-31535,whatever decide make sure make happy,False,False,False,False,True,True,True,False,False,False,False
2,2,2017-En-21068,max kellerman also help majority nfl coaching ...,True,False,True,False,True,False,True,False,False,False,False
3,3,2017-En-31436,accept challenge literally even feel exhilarat...,False,False,False,False,True,False,True,False,False,False,False
4,4,2017-En-22195,roommate okay spell autocorrect terrible first...,True,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6833,833,2017-En-21383,nicky57672 hi working towards highlighted conc...,False,False,False,False,False,False,False,False,False,False,False
6834,834,2017-En-41441,andreamitchell said berniesanders play hrc cam...,False,True,False,False,False,False,False,False,False,True,False
6835,835,2017-En-10886,isthataspider dhodgs fight guy insult lion lik...,True,False,True,False,False,False,False,True,False,False,False
6836,836,2017-En-40662,wonder guy broke penis sex serious,False,False,False,False,False,False,False,False,False,True,False


In [23]:
x_train = train['Tweet']
y_train = train[['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']]

x_val = val['Tweet']
y_val = val[['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']]

x_test = test['Tweet']
y_test = test[['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']]

**One hot Encoding**

In [24]:
train_label_encoded = pd.get_dummies(y_train.astype(int))
train = train.drop(['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'],  axis = 'columns')
train = pd.concat([train,train_label_encoded], axis = 'columns')

val_label_encoded = pd.get_dummies(y_val.astype(int))
val = val.drop(['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'],  axis = 'columns')
val = pd.concat([val,val_label_encoded], axis = 'columns')

test_label_encoded = pd.get_dummies(y_test.astype(int))
test = test.drop(['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'],  axis = 'columns')
test = pd.concat([test,test_label_encoded], axis = 'columns')

In [25]:
test

,Unnamed: 0,ID,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,0,2018-En-01559,adnan 786 asyounotwish dont worry indian army ...,1,1,0,0,0,0,1,0,0,0,1
1,1,2018-En-03739,academy science eschews normally sober tone sc...,0,0,1,0,0,0,0,0,0,0,0
2,2,2018-En-00385,blew opportunity mad,1,0,1,0,0,0,0,0,1,0,0
3,3,2018-En-03001,time 2 week 30,0,0,0,0,1,0,0,0,1,0,0
4,4,2018-En-01988,deppression real partner w depressed people tr...,0,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3254,254,2018-En-03848,shaft abrasion panty merely shifted side n n t...,1,0,0,0,0,0,0,1,0,0,0
3255,255,2018-En-00416,lomadia heard remothered indie horror game wri...,0,1,0,0,0,0,0,1,0,0,0
3256,256,2018-En-03717,fake outrage need stop,1,0,1,0,0,0,0,0,0,0,0
3257,257,2018-En-03504,would ever grateful could record garden forgiv...,0,0,0,0,1,0,0,0,0,0,0


In [26]:
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(x_train)
X_val_vectorized = vectorizer.transform(x_val)
X_test_vectorized = vectorizer.transform(x_test)

In [27]:
from sklearn.svm import SVC

# Create an SVM classifier with a linear kernel
svm_classifier = SVC(kernel='linear', probability=True)

# Wrap the classifier with OneVsRestClassifier
ovr_classifier = OneVsRestClassifier(svm_classifier)

# Train the model
ovr_classifier.fit(X_train_vectorized, y_train)

# Adjust the hyperparameter grid for the SVM
param_grid = {
    'estimator__C': [0.1, 1, 10],  # Example values for C
    'estimator__kernel': ['linear']  # Only 'linear' is needed since we already chose a linear kernel above
}

# Use accuracy as the scoring metric for GridSearchCV
scorer = make_scorer(accuracy_score)

# Create a GridSearchCV object
grid_search = GridSearchCV(ovr_classifier, param_grid, scoring=scorer, cv=5)

# Fit the model on the training set
grid_search.fit(X_train_vectorized, y_train)

# Get the best hyperparameters from the grid search
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

# Make predictions on the validation set using the best hyperparameters
val_predictions = grid_search.predict(X_val_vectorized)

# Evaluate the model on the validation set
val_accuracy = accuracy_score(y_val, val_predictions)
print(f"Validation Accuracy with Best Hyperparameters: {val_accuracy:.2f}")

# Make predictions on the test set using the best hyperparameters
test_predictions = grid_search.predict(X_test_vectorized)

# Evaluate the model on the test set
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Test Accuracy with Best Hyperparameters: {test_accuracy:.2f}")

# Print classification report for the test set
print("Classification Report for Test Set:")
print(classification_report(y_test, test_predictions))

Best parameters: {'estimator__C': 1, 'estimator__kernel': 'linear'}
Validation Accuracy with Best Hyperparameters: 0.19
Test Accuracy with Best Hyperparameters: 0.20
Classification Report for Test Set:
              precision    recall  f1-score   support

           0       0.74      0.59      0.66      1101
           1       0.50      0.02      0.04       425
           2       0.68      0.55      0.61      1099
           3       0.83      0.55      0.66       485
           4       0.85      0.66      0.74      1442
           5       0.75      0.31      0.44       516
           6       0.70      0.48      0.57      1143
           7       0.67      0.06      0.11       375
           8       0.79      0.40      0.53       960
           9       0.45      0.05      0.09       170
          10       0.00      0.00      0.00       153

   micro avg       0.76      0.46      0.57      7869
   macro avg       0.63      0.33      0.40      7869
weighted avg       0.72      0.46      0

c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifie